In [ ]:
from Enceladus.jobs import TrainingPipeline
from Enceladus.utils import GetConfiguration

pipeline, model, sweep = GetConfiguration().run('config.ini')
worker = TrainingPipeline(
    config=pipeline,
    model_config=model,
    sweep_config=sweep,
    no_sweep=True,
)
worker.run()

In [ ]:
import keras
import numpy as np
import tensorflow as tf
from Enceladus.utils import RecordsHandler

model = keras.models.load_model('enceladus_model')

In [ ]:
handler = RecordsHandler(data_dir='/home/cam/Documents/database_tools/data-2022-10-28/mimic3/')
dataset = handler.read_records(n_cores=10, AUTOTUNE=tf.data.AUTOTUNE)
test = dataset['test']

ppg = []
abp = []
for p, a in test.take(10):
    ppg.append(p)
    abp.append(a)
ppg = np.array(ppg)
abp = np.array(abp)

In [ ]:
model.predict(ppg).shape